In [21]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('/output_left.csv')

# Fill NaN values with placeholders
df.fillna('0.0.0.0', inplace=True)  # For IP addresses
df.fillna('00:00:00:00:00:00', inplace=True)  # For MAC addresses

# Drop IP, MAC address, and address columns
df.drop(columns=['sMACs', 'rMACs', 'sIPs', 'rIPs', 'sAddress', 'rAddress'], inplace=True)

# Convert datetime columns to appropriate numeric representations
df['startDate'] = pd.to_datetime(df['startDate']).astype(int) // 10**9  # Convert to Unix timestamp

# Encode 'protocol' column
le = LabelEncoder()
df['protocol'] = le.fit_transform(df['protocol'])

# Select features and target (assuming 'NST_B_Label' is the target for binary classification)
features = df.drop(columns=['IT_B_Label', 'IT_M_Label', 'NST_B_Label', 'NST_M_Label'])
target = df['NST_B_Label']  #'NST_B_Label' for binary anomaly detection

# Ensure all columns are numeric
numeric_features = features.select_dtypes(include=['number'])

# Standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(numeric_features)

# Convert back to a DataFrame
features_scaled_df = pd.DataFrame(features_scaled, columns=numeric_features.columns)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled_df, target, test_size=0.1, random_state=10)

# Display the shapes of the resulting datasets
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')


X_train shape: (7699, 15), y_train shape: (7699,)
X_test shape: (856, 15), y_test shape: (856,)


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Initialize the Random Forest classifier
clf = RandomForestClassifier(random_state=42)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Predict on the testing data
y_pred = clf.predict(X_test)

# Evaluate the model
'''print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))'''



'print("Classification Report:")\nprint(classification_report(y_test, y_pred))\n\n# Confusion matrix\nprint("Confusion Matrix:")\nprint(confusion_matrix(y_test, y_pred))'

In [13]:
import joblib
joblib.dump(clf, 'trained_model.pkl')

['trained_model.pkl']

In [22]:
import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('/content/output_left.csv')

# Fill NaN values with placeholders
df.fillna('0.0.0.0', inplace=True)  # For IP addresses
df.fillna('00:00:00:00:00:00', inplace=True)  # For MAC addresses

# Drop IP, MAC address, and address columns
df.drop(columns=['sMACs', 'rMACs', 'sIPs', 'rIPs', 'sAddress', 'rAddress'], inplace=True)

# Convert datetime columns to appropriate numeric representations
df['startDate'] = pd.to_datetime(df['startDate']).astype(int) // 10**9  # Convert to Unix timestamp

# Encode 'protocol' column
le = LabelEncoder()
df['protocol'] = le.fit_transform(df['protocol'])

expected_features = [
    'protocol','startDate','start','end','startOffset','endOffset','duration',  'sPackets', 'rPackets', 'sBytesSum', 'rBytesSum',
    'sLoad','rLoad','sPayloadSum','rPayloadSum'
]
ground_truth = df['IT_B_Label']
# Ensure df only contains the expected features
df = df[expected_features]

# Load the trained model
model = joblib.load('/content/trained_model.pkl')

# Make predictions on new data
predictions = model.predict(df)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ground_truth, predictions)
print("Accuracy of the model on new data:", accuracy)




Accuracy of the model on new data: 0.7807130333138516
